In [ ]:
# This is currently unused as it doesn't really work

from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from pyserini.search import LuceneSearcher
import torch

# Load DPR (Dense Passage Retrieval) model and tokenizer
model_name = 'facebook/dpr-ctx_encoder-single-nq-base'
tokenizer = DPRContextEncoderTokenizer.from_pretrained(model_name)
model = DPRContextEncoder.from_pretrained(model_name)
searcher = LuceneSearcher('../drive/MyDrive/UW/CSE 447/Final Project/NLP/sample_collection_jsonl/')

def getDocsForQuestion(question):
# Define your query and list of passages
  hits = searcher.search(question, 100)

  formatted_hits = [json.loads(searcher.doc(hit.docid).raw())['contents'] for hit in hits]
  query = question
  passages = formatted_hits

  print(passages)

  # Tokenize query and passages
  inputs = tokenizer([query]*len(passages), passages, return_tensors='pt', padding=True, truncation=True)

  # Encode query and passages
  with torch.no_grad():
      outputs = model(**inputs)

  # Extract passage embeddings from outputs
  passage_embeddings = outputs.pooler_output.cpu().numpy()

  # Compute similarity scores between query and passages
  query_embedding = passage_embeddings[0]  # Extract query embedding
  passage_embeddings = passage_embeddings[1:]  # Extract passage embeddings
  similarity_scores = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding).unsqueeze(0), torch.tensor(passage_embeddings), dim=1)

  # Sort passages based on similarity scores (higher scores mean higher relevance)
  sorted_passages = sorted(zip(passages, similarity_scores), key=lambda x: x[1], reverse=True)

  # Print reranked passages
  for passage, score in sorted_passages:
      print(f"Passage: {passage}\nSimilarity Score: {score}\n")

In [ ]:
docs = getDocsForQuestion('What language is used in Saint Vincent and the Grenadines?', 
                          "../drive/MyDrive/UW/CSE 447/Final Project/NLP/sample_collection_jsonl/")

print(docs)